In [91]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sklearn
from scipy import ndimage, misc

%matplotlib inline
import cv2
import glob

img_list = []
filenamelist= []
for img in glob.glob("Stop1_1000/*.jpg"):
    #img_list.append(cv2.imread(img,0))
    filenamelist.append(img)
np.array(filenamelist).shape

(1000,)

In [92]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center < (radius+10)
    return mask

In [93]:
def mask_innercicle(img):
    
    ret, thresh1 = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)
    _ , contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    import circle_fit as cf

    circle_point = []

    for i in range(np.array(contours).shape[0]):
            if(np.array(contours[i]).shape[0]>1500):
                #print(np.array(contours[i]).shape)

                cor = np.array(contours[i])
                cor = cor.reshape(cor.shape[0],cor.shape[2])
                #print(cor.shape)

                xc,yc,r,_ = cf.least_squares_circle(cor)
                print(xc,yc,r)
                circle_point.append([xc,yc,r])

    cor = np.array(contours[1])
    cor = cor.reshape(cor.shape[0],cor.shape[2])

    xc,yc,r,_ = cf.least_squares_circle(cor)

    h = img.shape[0]
    w = img.shape[1]
    center = (circle_point[1][0].astype(int), circle_point[1][1].astype(int))
    radius = circle_point[1][2].astype(int)
    mask = create_circular_mask(h, w, center=center, radius=radius)
    
    mask_array = np.ma.masked_array(img, mask=mask,fill_value = 255)
    img_temp = mask_array.filled()
    return img_temp

In [94]:
for iter_file in range(len(filenamelist)):
    filename = filenamelist[iter_file]
    print(filename)
    img = np.array(cv2.imread(filename,0))
    

    img_temp = mask_innercicle(img)
    
    
    #img_temp = cv2.blur(img_temp,(3,3))
    
    #img_temp = ndimage.minimum_filter(img_temp, size=3)
    #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(3,3))#clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
    #img_temp = clahe.apply(img_temp)
    #img_temp = cv2.blur(img_temp,(3,3))
   
    vis = img_temp.copy()
    vis_rgb = cv2.cvtColor(vis,cv2.COLOR_GRAY2RGB)    
    # local MSER
    for i in range(2):
        if i==0 :
            mser = cv2.MSER_create(_delta=5, _min_area=800,_max_variation=0.7)
        else:
            mser = cv2.MSER_create(_delta=5, _min_area=120, _max_area = 800, _max_variation=1.2)
            #mser = cv2.MSER_create(_delta=6, _min_area=120, _max_area = 800, _max_variation=1.6)
        #mser = cv2.MSER_create(_delta=4, _min_area=150,_max_variation=0.5)
        #mser = cv2.MSER_create(_delta=3, _min_area=150,_max_variation=0.25)
        #cv2.MSER_create(_delta=3, _min_area=100,_max_variation=0.3)
        regions = mser.detectRegions(img_temp)


        hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions[0]]
        
        #print(hulls)
        #print(np.array(hulls).shape)
        mask = np.zeros_like(vis)
        #print(np.array(hulls).shape[0])
        for i in range(np.array(hulls).shape[0]):
            mask_now = cv2.fillPoly(mask, [hulls[i]],1)
            mask_now = vis*mask_now
            min_in_poly = np.min(mask_now[mask_now>0])
            mean_in_poly = np.mean(mask_now[mask_now>0])
            #print(min_in_poly , " ", mean_in_poly)
            #print(min_in_poly)
            if(min_in_poly>100 or mean_in_poly>130):
                continue
            
            cv2.polylines(vis_rgb, [hulls[i]], 1, (255, 0, 0))

    
    print(".\\Detect\\" +filenamelist[iter_file][11:-4]+'_Detect.jpg')
    cv2.imwrite(".\\Detect\\" +filenamelist[iter_file][11:-4]+'_Detect.jpg',vis_rgb)
    plt.close()

Stop1_1000\1.jpg
1102.2156428492951 1102.1958702081997 758.9927298003878
1103.8155534788764 1102.9755961377564 642.3043755005046
.\Detect\1_Detect.jpg
Stop1_1000\10.jpg
1105.162124940649 1097.0700256057187 758.1608258472824
1102.6384358810446 1095.8735624614744 641.2404835000697
.\Detect\10_Detect.jpg
Stop1_1000\100.jpg
1112.1788312069798 1087.7182769851368 761.0764791698007
1113.1837837705377 1088.7158856877138 645.6989211525726
.\Detect\100_Detect.jpg
Stop1_1000\1000.jpg
1122.5730375327364 1072.4715946991516 760.7335905716393
1125.083731589087 1070.8927224942609 645.6862252387145
.\Detect\1000_Detect.jpg
Stop1_1000\101.jpg
1105.7427082211173 1092.3443831441004 759.653584753961
1107.796767721839 1093.6002560524132 644.5298118473597
.\Detect\101_Detect.jpg
Stop1_1000\102.jpg
1118.4373261715332 1083.1993490439552 760.5571113841427
1117.4439751635437 1083.2733389844254 645.2559205511513
.\Detect\102_Detect.jpg
Stop1_1000\103.jpg
1099.3936351340387 1095.936078029798 757.5373118340927
1101

.\Detect\146_Detect.jpg
Stop1_1000\147.jpg
1111.3571012033494 1088.5294793219657 763.5937235793258
1112.1593530576426 1088.0837795532207 648.1009061825272
.\Detect\147_Detect.jpg
Stop1_1000\148.jpg
1115.7681009553874 1082.296074655821 762.4525362506057
1115.8216157159786 1082.1494157353716 647.055856810904
.\Detect\148_Detect.jpg
Stop1_1000\149.jpg
1095.5009442398757 1101.7776335417411 758.7760772180865
1097.5988141562848 1101.9986257353894 642.7010326880641
.\Detect\149_Detect.jpg
Stop1_1000\15.jpg
1106.0313746716934 1089.8600177981557 760.9981049990226
1106.1235799585527 1086.9416536141343 642.9414215592227
.\Detect\15_Detect.jpg
Stop1_1000\150.jpg
1117.285267935427 1086.817082685074 758.067837518543
1117.1771647661499 1088.375140436786 642.953696534509
.\Detect\150_Detect.jpg
Stop1_1000\151.jpg
1110.037202459215 1090.2251326252972 758.750089641543
1112.9620018381654 1088.0391078167197 642.6470824805169
.\Detect\151_Detect.jpg
Stop1_1000\152.jpg
1109.9017889523543 1089.4768941658313 

.\Detect\195_Detect.jpg
Stop1_1000\196.jpg
1112.0681805937954 1089.3839692696733 760.4963879006291
1115.938309400021 1089.0618464129486 645.0565667901182
.\Detect\196_Detect.jpg
Stop1_1000\197.jpg
1119.0112519889187 1078.4404506982482 761.0521748122425
1120.0962141897055 1079.250148819461 645.7550465822302
.\Detect\197_Detect.jpg
Stop1_1000\198.jpg
1108.7363402620056 1092.8617412681917 759.58112097338
1109.5488540143199 1090.518219744533 643.4003880868044
.\Detect\198_Detect.jpg
Stop1_1000\199.jpg
1104.0325135712121 1085.9519405003045 759.3936223345728
1103.1220982080456 1083.5434975230544 643.9464283788963
.\Detect\199_Detect.jpg
Stop1_1000\2.jpg
1102.336898546905 1094.0279707588509 759.6044983975382
1104.6022405937347 1092.4095768219206 642.8729812756612
.\Detect\2_Detect.jpg
Stop1_1000\20.jpg
1101.5308706280748 1106.8971019060634 761.1685243050882
1101.733668480702 1106.9907675007657 646.2185392266275
.\Detect\20_Detect.jpg
Stop1_1000\200.jpg
1110.4503041486432 1091.7027160731272 76

.\Detect\243_Detect.jpg
Stop1_1000\244.jpg
1095.8008394156666 1105.853669626561 762.0073683624189
1096.228024735469 1106.4945992026983 646.6563429941863
.\Detect\244_Detect.jpg
Stop1_1000\245.jpg
1102.8234658888648 1088.031380017174 761.197292383383
1101.8836382974853 1086.574351537256 642.5666368447655
.\Detect\245_Detect.jpg
Stop1_1000\246.jpg
1103.0359931777107 1088.1062816028088 759.862728939516
1102.7669698302109 1088.7102952977198 644.4138696230212
.\Detect\246_Detect.jpg
Stop1_1000\247.jpg
1102.8052816175061 1096.215352020043 761.0240827120699
1103.2280448546749 1098.112914143654 645.6441552116827
.\Detect\247_Detect.jpg
Stop1_1000\248.jpg
1113.511987882333 1091.8895881875842 761.6809172660338
1113.5166609563446 1093.345954062525 647.1933393777127
.\Detect\248_Detect.jpg
Stop1_1000\249.jpg
1108.6569429782076 1101.090668381199 761.229677284564
1110.374877706824 1101.214646709597 646.6329988923922
.\Detect\249_Detect.jpg
Stop1_1000\25.jpg
1119.4930581688036 1077.6062089275285 758.

.\Detect\292_Detect.jpg
Stop1_1000\293.jpg
1095.0507872269743 1109.0393498399667 759.9511680234394
1094.4627948154316 1109.7724229313121 644.000360174934
.\Detect\293_Detect.jpg
Stop1_1000\294.jpg
1119.3118052144514 1083.0634866448088 761.9150783006567
1120.606027847334 1082.5205474901472 644.813948247937
.\Detect\294_Detect.jpg
Stop1_1000\295.jpg
1097.3418808179115 1103.3464489620035 759.5775999884048
1099.8300638976655 1104.1664690436728 644.0583497878713
.\Detect\295_Detect.jpg
Stop1_1000\296.jpg
1109.6312645771318 1086.8596419545995 758.1672096789666
1113.1396328401715 1088.8053582489445 641.5372027316118
.\Detect\296_Detect.jpg
Stop1_1000\297.jpg
1118.340085618409 1083.9627724593938 759.5676396221619
1119.0465009424913 1083.8735158522284 645.6000742280372
.\Detect\297_Detect.jpg
Stop1_1000\298.jpg
1122.736514862824 1076.3671098716557 760.561409340277
1124.0581739232616 1076.8296316906665 645.7372528432309
.\Detect\298_Detect.jpg
Stop1_1000\299.jpg
1104.2206337006644 1103.913353323

...

.\Detect\779_Detect.jpg
Stop1_1000\78.jpg
1093.9567977287559 1107.6293651472633 758.6366298471219
1098.1296109588866 1107.7417685082853 643.1334099136207
.\Detect\78_Detect.jpg
Stop1_1000\780.jpg
1106.4913802190717 1088.5736617994537 759.8739667072812
1106.8688480543476 1087.6915336664795 644.7639356972337
.\Detect\780_Detect.jpg
Stop1_1000\781.jpg
1114.7341645608005 1064.9891427543027 758.8039243880314
1116.3115760381913 1069.4298844203433 642.2110642384488
.\Detect\781_Detect.jpg
Stop1_1000\782.jpg
1103.6148040753042 1096.6786233354158 758.6474862871992
1104.764643518046 1095.1092621610571 644.7537065331903
.\Detect\782_Detect.jpg
Stop1_1000\783.jpg
1126.1372023794281 1064.4767912614436 760.1555891682203
1130.3793951322 1063.8043816255367 639.7164808244974
.\Detect\783_Detect.jpg
Stop1_1000\784.jpg
1118.5284036968594 1083.8061420430022 761.1568936300177
1119.4696647588992 1084.1721795121393 644.0524451943347
.\Detect\784_Detect.jpg
Stop1_1000\785.jpg
1106.8982849132715 1083.039888412

.\Detect\827_Detect.jpg
Stop1_1000\828.jpg
1107.1680895585007 1099.8488794098268 761.2579979621075
1106.6174404137914 1101.0083644093595 645.5526639891093
.\Detect\828_Detect.jpg
Stop1_1000\829.jpg
1110.5978856878498 1085.9672863379938 760.0303586221428
1113.5680926709051 1084.3719760889892 643.312739182132
.\Detect\829_Detect.jpg
Stop1_1000\83.jpg
1107.7856303580434 1092.3677091460586 758.8612486358552
1109.7470576856704 1093.5293316449297 644.574680541409
.\Detect\83_Detect.jpg
Stop1_1000\830.jpg
1114.3462987324588 1086.3294509289883 761.2116180694196
1115.364002318523 1085.820144131107 645.4151079127394
.\Detect\830_Detect.jpg
Stop1_1000\831.jpg
1103.4946658327756 1096.8840604352501 758.8642374302908
1106.0138514770924 1096.4211898198002 642.9944683694055
.\Detect\831_Detect.jpg
Stop1_1000\832.jpg
1086.4244558663888 1107.4638932512541 757.7739513085384
1089.3248798122643 1108.4356359348096 642.2496860887569
.\Detect\832_Detect.jpg
Stop1_1000\833.jpg
1106.667122349623 1084.3128404220

.\Detect\875_Detect.jpg
Stop1_1000\876.jpg
1113.501352186667 1094.9082427231942 760.7428648607469
1113.560870798155 1095.3569288356757 644.1280056806063
.\Detect\876_Detect.jpg
Stop1_1000\877.jpg
1097.324990690397 1099.1978498226279 759.6678509442931
1096.961066442027 1099.5634102776492 641.5797847861411
.\Detect\877_Detect.jpg
Stop1_1000\878.jpg
1104.9541734594534 1102.2564931929608 761.3051882948848
1108.9323400179733 1101.374301204685 642.6677211730155
.\Detect\878_Detect.jpg
Stop1_1000\879.jpg
1118.7508405019819 1084.5277913772884 761.4872052331942
1119.1416399753507 1084.5814090947297 646.7106997243529
.\Detect\879_Detect.jpg
Stop1_1000\88.jpg
1104.599719812264 1101.573399343023 758.1006308209403
1104.3075814907033 1100.771156442665 643.3469035117183
.\Detect\88_Detect.jpg
Stop1_1000\880.jpg
1114.7724676761968 1084.314646701169 761.8572663561442
1119.0759262115644 1084.6898252385618 644.1514319535302
.\Detect\880_Detect.jpg
Stop1_1000\881.jpg
1111.0792930227715 1084.2838260876945 

.\Detect\923_Detect.jpg
Stop1_1000\924.jpg
1110.746496453885 1095.9143018069835 759.8714902325775
1110.261440580682 1095.8815489232406 643.8598782754934
.\Detect\924_Detect.jpg
Stop1_1000\925.jpg
1111.2185239098612 1091.1448492953898 760.1974604142873
1111.1302904363865 1090.8270496726684 644.9171506837964
.\Detect\925_Detect.jpg
Stop1_1000\926.jpg
1109.467464606186 1085.0085447933584 760.3581259861531
1108.3424274029765 1086.7468334481807 641.8883168582372
.\Detect\926_Detect.jpg
Stop1_1000\927.jpg
1110.8102781759974 1087.3308802511838 759.9285024824383
1116.4763717498574 1087.5306468233098 641.1617399751151
.\Detect\927_Detect.jpg
Stop1_1000\928.jpg
1114.7242677902493 1096.334616391503 759.7190084052226
1117.8150582563976 1093.7637501235904 642.9100572217897
.\Detect\928_Detect.jpg
Stop1_1000\929.jpg
1113.802835748137 1089.9518434398276 760.525360467821
1115.7162272370758 1092.1511496650455 644.127811703609
.\Detect\929_Detect.jpg
Stop1_1000\93.jpg
1105.9715351621517 1096.98117710690

.\Detect\972_Detect.jpg
Stop1_1000\973.jpg
1105.9301304944668 1091.190266470136 760.4687482556877
1109.125993007629 1093.528413797114 642.785173884807
.\Detect\973_Detect.jpg
Stop1_1000\974.jpg
1104.5312129177019 1092.5652805096158 760.3765718845708
1105.9877522601394 1093.7583494881376 643.9831383916995
.\Detect\974_Detect.jpg
Stop1_1000\975.jpg
1115.5714710612574 1087.8075772386603 760.994614485305
1117.9080825513756 1087.140166913092 644.885790842947
.\Detect\975_Detect.jpg
Stop1_1000\976.jpg
1106.4442568142333 1100.003842366642 761.06353318413
1109.5739593546134 1098.7582197187774 644.3371228941108
.\Detect\976_Detect.jpg
Stop1_1000\977.jpg
1107.7816030776878 1090.4042622740355 760.5261967923259
1107.0304104436102 1089.7144675111372 645.2753074236289
.\Detect\977_Detect.jpg
Stop1_1000\978.jpg
1105.2574817352067 1095.321180312309 760.9823506414943
1104.9292238377855 1094.4396722436388 645.9973602735886
.\Detect\978_Detect.jpg
Stop1_1000\979.jpg
1099.865684003526 1099.3460149859125 7

### 筆記
123.jpg
124.jpg

因為沒有拍到全圓所以有點問題